# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 
import time
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv("output_data/cities.csv")
city_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Clouds,Wind,Country,Date
0,0,vaini,-21.2000,-175.2000,71.76,78,40,12.66,TO,1624684009
1,1,hay river,60.8156,-115.7999,59.94,65,62,3.87,CA,1624684009
2,2,ushuaia,-54.8000,-68.3000,38.86,75,75,25.32,AR,1624684010
3,3,zolotukhino,52.0842,36.3772,77.07,75,45,4.54,RU,1624684010
4,4,yar-sale,66.8333,70.8333,56.59,40,10,8.01,RU,1624684011


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [4]:
location = city_df[["Latitude", "Longitude"]]
humidity = city_df["Humidity"]

In [5]:
fig = gmaps.figure()

heat_map = gmaps.heatmap_layer(
    location, 
    weights=city_df["Humidity"],
    dissipating=False,
    max_intensity=200,
    point_radius = 3)
fig.add_layer(heat_map)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
perfect_city_df = city_df.loc[(city_df["Wind"]<= 10) & (city_df["Clouds"] <= 0) & 
                                (city_df["Max Temperature"]>= 70) & (city_df["Max Temperature"]<= 80)].dropna()

perfect_city_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Clouds,Wind,Country,Date
79,79,alexandria,31.2156,29.9553,78.75,72,0,6.06,EG,1624684037
186,186,abu samrah,35.3029,37.1841,79.50,45,0,6.38,SY,1624684074
194,194,panjakent,39.4952,67.6093,78.60,24,0,5.99,TJ,1624684077
219,219,saint-pierre,-21.3393,55.4781,72.57,68,0,4.61,RE,1624684085
247,247,pomorie,42.5588,27.6439,75.63,88,0,0.00,BG,1624684094
522,522,avera,33.1940,-82.5271,73.06,60,0,5.82,US,1624684193
543,543,mbandaka,0.0487,18.2603,70.20,88,0,2.35,CD,1624684201


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = perfect_city_df[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
79,alexandria,EG,31.2156,29.9553,
186,abu samrah,SY,35.3029,37.1841,
194,panjakent,TJ,39.4952,67.6093,
219,saint-pierre,RE,-21.3393,55.4781,
247,pomorie,BG,42.5588,27.6439,
522,avera,US,33.1940,-82.5271,
543,mbandaka,CD,0.0487,18.2603,


In [8]:
lat = hotel_df.groupby('City')['Latitude']
#lng = perfect_city_df.loc["Longitude"]
lat.head()

79     31.2156
186    35.3029
194    39.4952
219   -21.3393
247    42.5588
522    33.1940
543     0.0487
Name: Latitude, dtype: float64

In [9]:
hotels = []

# (hotel_df["City"].unique()):

    # lat = perfect_city_df.groupby('City')["Latitude"]
    # lng = perfect_city_df.groupby('City')["Longitude"]
    # lat = perfect_city_df.loc[perfect_city_df["City"]==city, ["Latitude"]]
    # lng = perfect_city_df.loc[perfect_city_df["City"]==city, ["Longitude"]]
    #city_coordinate = f"{lat},{lng}"
for index, row in hotel_df.iterrows():

    lat = row["Latitude"]
    lng = row["Longitude"]

    params = {
        "location": f"{lat},{lng}",
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

hotel_df["Hotel Name"] = hotels
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
79,alexandria,EG,31.2156,29.9553,Royal Crown Hotel
186,abu samrah,SY,35.3029,37.1841,Nearest hotel not found
194,panjakent,TJ,39.4952,67.6093,Salom Hostel
219,saint-pierre,RE,-21.3393,55.4781,Lindsey Hôtel
247,pomorie,BG,42.5588,27.6439,Spa Hotel St. George
522,avera,US,33.1940,-82.5271,Nearest hotel not found
543,mbandaka,CD,0.0487,18.2603,Guest House Epervier


In [10]:
lat = perfect_city_df.loc[perfect_city_df["City"]=="alexandria", ["Latitude"]]
lat

,Latitude
79,31.2156


In [11]:
(hotel_df["City"].unique())

array(['alexandria', 'abu samrah', 'panjakent', 'saint-pierre', 'pomorie',
       'avera', 'mbandaka'], dtype=object)

In [12]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": f"-33.93,18.42"
}
    # Use the search term: "Hotel" and our lat/lng
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
name_address = requests.get(base_url, params=params).json()
    

In [13]:
name_address

{'html_attributions': [],
 'next_page_token': 'Aap_uEA3StVuOXHtUUpyQTvs0_Cy9uyXrTsPrMEVIW7q-Yy5BwHyUzk93bDKCxR3KxGur6Q8-U13PsPe3Q9uCzu_MDxkuUvzdVJ3ov1_VbgnOilNwLdWkFc8Wc-0FNK1RQ9mDs7KqIcbmfh9PT3HNYIr4cPESyItj5wEGxpMd8WPA29osIas0Q-SmJaCgAoJP10xXPQJkqGY95Nbo_EldEUwsecIjqmOkGCkYlFXY64I42QVTtnrlbcV7zi43F6meIA7JhbiDgOOdMuM2pRThK11IQvV4ugmyBf77eUiGIQsaSTIm-Nc8yHH8Jl00OheUaBYezmMQPgLdzKpQLrXyjTU-iARxsTBW6eD6wcHgjlq5pgIglMX15J7lYrd_UJz98F-9YfEOBfUsWB2sCEZ2OPwHPYGzblbCIVt5V8fcfkoQru6JIQerJywIhoC',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': -33.9214218, 'lng': 18.4220856},
    'viewport': {'northeast': {'lat': -33.9200460697085,
      'lng': 18.4234635302915},
     'southwest': {'lat': -33.9227440302915, 'lng': 18.4207655697085}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'Southern Sun Cape Sun',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 608,
     'html_attributions': ['<a h

In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [15]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))